In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_sla = pd.read_excel('../input/logistics-shopee-code-league/SLA_matrix.xlsx')
df_order = pd.read_csv('../input/logistics-shopee-code-league/delivery_orders_march.csv')
df_sla.head(10)

In [ ]:
route = [i for i in df_sla.iloc[0].dropna()]

In [ ]:
dict(enumerate(route))

In [ ]:
df_sla['Destination (Buyer)'] = df_sla['Destination (Buyer)'].iloc[1:5].str.extract(pat = '(1|2|3|4|5|6|7|8|9)')
df_sla['Unnamed: 3'] = df_sla['Unnamed: 3'].iloc[1:5].str.extract(pat = '(1|2|3|4|5|6|7|8|9)')
df_sla['Unnamed: 4'] = df_sla['Unnamed: 4'].iloc[1:5].str.extract(pat = '(1|2|3|4|5|6|7|8|9)')
df_sla['Unnamed: 5'] = df_sla['Unnamed: 5'].iloc[1:5].str.extract(pat = '(1|2|3|4|5|6|7|8|9)')

In [ ]:
df_sla

In [ ]:
sla = df_sla.iloc[1:5,2:6]
sla.columns = route
sla.rename(index = dict(enumerate(route,1)), inplace=True)
sla.head()

In [ ]:
sla.loc['Metro Manila']

In [ ]:
df_order.head(10)

In [ ]:
df_order['pick'] = pd.to_datetime(df_order['pick'],unit='s')
df_order['1st_deliver_attempt'] = pd.to_datetime(df_order['1st_deliver_attempt'],unit='s')
df_order['2nd_deliver_attempt'] = pd.to_datetime(df_order['2nd_deliver_attempt'],unit='s')
df_order

In [ ]:
df_order.pick = df_order.pick.dt.tz_localize('UTC').dt.tz_convert('Asia/Manila')
df_order['1st_deliver_attempt'] = df_order['1st_deliver_attempt'].dt.tz_localize('UTC').dt.tz_convert('Asia/Manila')
df_order['2nd_deliver_attempt'] = df_order['2nd_deliver_attempt'].dt.tz_localize('UTC').dt.tz_convert('Asia/Manila')

In [ ]:
df_order

In [ ]:
df_order.pick = df_order.pick.dt.date
df_order['1st_deliver_attempt'] = df_order['1st_deliver_attempt'].dt.date
df_order['2nd_deliver_attempt'] = df_order['2nd_deliver_attempt'].dt.date
df_order

In [ ]:
df_order['oneattempt'] = df_order['1st_deliver_attempt'] - df_order['pick']
df_order

In [ ]:
df_order['2nd_deliver_attempt'].isnull()

In [ ]:
df_order['twoattempt'] = df_order['2nd_deliver_attempt'] - df_order['1st_deliver_attempt']
df_order

In [ ]:
(df_order['buyeraddress'].str.contains('Metro|Manila|Luzon|Visayas|Mindanao|metro|manila|luzon|visayas|mindanao', 
                                       regex=True)).unique()

In [ ]:
(df_order['selleraddress'].str.contains('Metro|Manila|Luzon|Visayas|Mindanao|metro|manila|luzon|visayas|mindanao', 
                                       regex=True)).unique()

In [ ]:
df_order['buyeraddress'] = df_order['buyeraddress'].str.capitalize().str.extract(pat = '(Metro|Manila|Luzon|Visayas|Mindanao)')
df_order['selleraddress'] = df_order['selleraddress'].str.capitalize().str.extract(pat = '(Metro|Manila|Luzon|Visayas|Mindanao)')

In [ ]:
df_order['buyeraddress'] = df_order['buyeraddress'].str.replace('Metro|Manila','Metro Manila')
df_order['selleraddress'] = df_order['selleraddress'].str.replace('Metro|Manila','Metro Manila')

In [ ]:
df_order.head()

In [ ]:
df_order['Estimate'] = [sla[df_order['buyeraddress'].iloc[i]][df_order['selleraddress'].iloc[i]] for i in range(len(df_order))]

In [ ]:
df_order.head()

In [ ]:
df_order['oneattempt'] = df_order['oneattempt'].dt.days

In [ ]:
df_order['twoattempt'] = df_order['twoattempt'].dt.days

In [ ]:
df_order = df_order.fillna(0)

In [ ]:
df_order.head()

In [ ]:
orderid = []
is_late = []
for i in range(len(df_order)):
    orderid.append(df_order['orderid'].iloc[i])
    if int(df_order['oneattempt'].iloc[i]) > int(df_order['Estimate'].iloc[i]):
        is_late.append(1)
    elif int(df_order['twoattempt'].iloc[i]) != 0 and int(df_order['twoattempt'].iloc[i]) > 3:
        is_late.append(1)
    else:
        is_late.append(0)
    

In [ ]:
df = pd.DataFrame({'orderid' : orderid, 
                   'is_late' : is_late})

In [ ]:
df

In [ ]:
df.to_csv(r'/kaggle/working/submission.csv', index = False)